In [1]:
import pandas as pd
import numpy as np

### Prepare results

In [2]:
result_heuristics = pd.read_csv('results/heuristics_output.csv', delimiter=';', index_col=False)
result_heuristics.head(5)

,DATASET_NAME,METHOD_NAME,BIN_NUMBER,BIN_CAPACITY,N,ELAPSED_TIME_MS
0,Scholl_2_N3W4B1R9.txt,Next_fit,23,1000,200,0.01488
1,Scholl_2_N3W4B1R9.txt,First_fit,24,1000,200,0.17931
2,Scholl_2_N3W4B1R9.txt,Best_fit,24,1000,200,0.17646
3,Scholl_2_N4W3B1R9.txt,Next_fit,76,1000,500,0.03610
4,Scholl_2_N4W3B1R9.txt,First_fit,75,1000,500,1.30240


In [3]:
result_genetic = pd.read_csv('results/genetic_algorithm_output.csv', sep=';')
result_genetic.drop(columns=['POP_SIZE' ,'GENERATIONS', 'MUTATION_RATE', 'TOURNAMENT_SIZE'], inplace=True)
result_genetic.head(5)

,DATASET_NAME,METHOD_NAME,BIN_NUMBER,BIN_CAPACITY,N,ELAPSED_TIME_MS
0,Scholl_2_N3W4B1R9.txt,Genetic_Algorithm,62,1000,200,16764.44
1,Scholl_2_N4W3B1R9.txt,Genetic_Algorithm,170,1000,500,46723.31
2,Scholl_2_N4W1B1R3.txt,Genetic_Algorithm,243,1000,500,48974.18
3,Scholl_2_N4W3B2R1.txt,Genetic_Algorithm,168,1000,500,45335.92
4,Scholl_2_N4W3B1R1.txt,Genetic_Algorithm,170,1000,500,46006.53


In [4]:
results = pd.concat([result_genetic, result_heuristics])
results.sample(5)

,DATASET_NAME,METHOD_NAME,BIN_NUMBER,BIN_CAPACITY,N,ELAPSED_TIME_MS
3840,Wäscher_Waescher_TEST0005.txt,Next_fit,32,10000,114,0.01380
2740,Scholl_1_N3C3W2_T.txt,First_fit,80,150,200,0.60099
1462,Scholl_1_N2C2W4_G.txt,First_fit,61,120,100,0.22037
1870,Scholl_1_N1C2W1_A.txt,First_fit,21,120,50,0.04002
2274,Scholl_1_N4C2W4_O.txt,Next_fit,374,120,500,0.03767


### Parse dataset name

In [5]:
results['CATEGORY'] = np.nan
for dataset_category in ['Falkenauer U', 'Falkenauer_T', 'Scholl_1', 'Scholl_2', 'Wäscher']:
    mask = results['DATASET_NAME'].str.contains(dataset_category)
    results.loc[mask, 'CATEGORY'] = dataset_category
    results.loc[mask, 'DATASET_NAME'] = results.loc[mask, 'DATASET_NAME'].str.replace(f'{dataset_category}_', '', regex=False)

C:\Users\kkita\AppData\Local\Temp\ipykernel_13940\2388275774.py:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Falkenauer U' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  results.loc[mask, 'CATEGORY'] = dataset_category


In [6]:
results.sample(5)

,DATASET_NAME,METHOD_NAME,BIN_NUMBER,BIN_CAPACITY,N,ELAPSED_TIME_MS,CATEGORY
207,N1W3B1R9.txt,Genetic_Algorithm,12,1000,50,4422.21000,Scholl_2
2210,N1C3W4_D.txt,Best_fit,22,150,50,0.04896,Scholl_1
362,N4W2B2R9.txt,Best_fit,104,1000,500,1.46592,Scholl_2
1346,Falkenauer_u250_09.txt,Genetic_Algorithm,149,150,250,48358.70000,Falkenauer U
2169,N4C3W2_D.txt,Next_fit,283,150,500,0.03547,Scholl_1


### Load solutions from BPPLIB

In [10]:
solutions = pd.read_excel('results/Solutions.xlsx', sheet_name=['Falkenauer', 'Scholl', 'Wäscher'], usecols=['Name', 'Best UB'])
solutions = pd.concat(list(solutions.values()))
solutions.rename(columns={'Name': 'DATASET_NAME', 'Best UB': 'SOLUTION'}, inplace=True)

In [11]:
solutions.sample(5)

,DATASET_NAME,SOLUTION
730,N1W1B2R0.txt,17
95,Falkenauer_t60_15.txt,20
721,N1W1B1R1.txt,18
696,N4C3W2_Q.txt,199
1152,N4W3B2R2.txt,71


### Merge results and solutions

In [12]:
results = pd.merge(results, solutions, on=['DATASET_NAME'])

In [16]:
results.isna().sum()

DATASET_NAME       0
METHOD_NAME        0
BIN_NUMBER         0
BIN_CAPACITY       0
N                  0
ELAPSED_TIME_MS    0
CATEGORY           0
SOLUTION           0
dtype: int64

In [17]:
results.sample(5)

,DATASET_NAME,METHOD_NAME,BIN_NUMBER,BIN_CAPACITY,N,ELAPSED_TIME_MS,CATEGORY
1039,N4W1B3R7.txt,First_fit,163,1000,500,3.32485,Scholl_2
3192,N2C3W2_G.txt,Next_fit,55,150,100,0.00787,Scholl_1
382,N3W1B1R8.txt,Genetic_Algorithm,85,1000,200,18396.25000,Scholl_2
2592,N3C2W2_A.txt,Next_fit,142,120,200,0.01506,Scholl_1
1787,N1C3W1_I.txt,Best_fit,17,150,50,0.04255,Scholl_1
